In [77]:
import json; import requests; import datetime; import time; import csv

In [78]:
def set_api_key():
    with open("dark_sky_api_key.txt", "r") as myfile:
        api_key = myfile.readlines()[0][:-1]
    return api_key

In [79]:
nyc = {'lat':'40.7128','long':'74.0060'}

In [80]:
chicago = {'lat':'41.8781','long':'87.6298'}

In [81]:
# takes a single date input in dd/mm/yy format and returns a unix timestamp
def date_string_to_unix(date_string):
    return (str(int(time.mktime(datetime.datetime.strptime(date_string, "%d/%m/%Y").timetuple()))))

In [82]:
# return a list of unixtimestamps, one for each day between d1 and d2 specified in the function body
def unix_dates_for_year(year):
    start_date = datetime.date(year, 1, 1)  # start date
    end_date = datetime.date(year, 12, 31)  # end date
    delta = end_date - start_date      # timedelta
    dates = [(start_date + datetime.timedelta(days=i)) for i in range(delta.days + 1)]
    dates = [x.strftime('%d/%m/%Y') for x in dates]
    unix_dates  = [date_string_to_unix(dates[x]) for x in range(len(dates))]
    return unix_dates

In [83]:
# takes a timestamp, a lat and long, and returns a url string for a darksky request
def url_for_day(unix_timestamp,latitude,longitude):
    api_key = set_api_key()
    base_url = 'https://api.darksky.net/forecast/'
    tail = '?exclude=currently,flags,hourly&units=si&moonPhase'
    url = "{:s}{:s}/{:s},{:s},{:s}{:s}".format(base_url,api_key,latitude,longitude,unix_timestamp,tail)
    return url

In [84]:
def set_filename(year):
    return "{:s}_weather_data.txt".format(str(year))

In [85]:
# creates a csv file and creates headers from keys of an api request
def initialize_csv_headers(api_response_for_day,year):
    json_data = (api_response_for_day).json()
    keys = [str(x) for x in json_data['daily']['data'][0].keys()]
    filename = set_filename(year)
    with open(filename, 'w') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(keys)

In [104]:
# appends values of api call to file created by initialize_csv_headers
def append_daily_data(api_response_for_day,year):
    json_data = (api_response_for_day).json()
    values = [str(x) for x in json_data['daily']['data'][0].keys()]
    filename = set_filename(year)
    with open(filename, 'a') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(values)

In [105]:
# given a year, and a dictonary with {coordinates:value}, creates a file with that year's weather data
def csv_weather_data_for_year(year,city_coord):
    lat = city_coord['lat']
    long = city_coord['long']
    unix_dates = unix_dates_for_year(year)
    initialize_csv_headers(requests.get(url_for_day(unix_dates[0],lat,long)),year)
    for x in range(len(unix_dates)):
       append_daily_data(requests.get(url_for_day(unix_dates[x],lat,long)),year)

In [106]:
csv_weather_data_for_year(2015,nyc)

KeyboardInterrupt: 

In [88]:
lat = nyc['lat']
long = nyc['long']
unix_dates = unix_dates_for_year(2015)

In [97]:
res = requests.get(url_for_day(unix_dates[1],lat,long))

In [98]:
res.text

'{"latitude":40.7128,"longitude":74.006,"timezone":"Asia/Bishkek","daily":{"data":[{"time":1420135200,"summary":"Foggy starting in the evening.","icon":"fog","sunriseTime":1420165757,"sunsetTime":1420199307,"moonPhase":0.41,"precipType":"rain","temperatureHigh":5,"temperatureHighTime":1420185600,"temperatureLow":-2.47,"temperatureLowTime":1420221600,"apparentTemperatureHigh":5,"apparentTemperatureHighTime":1420192800,"apparentTemperatureLow":-4.99,"apparentTemperatureLowTime":1420232400,"dewPoint":-4.48,"humidity":0.68,"pressure":1020.47,"windSpeed":0.9,"windBearing":192,"cloudCover":0.19,"visibility":5.62,"temperatureMin":-3.54,"temperatureMinTime":1420156800,"temperatureMax":5,"temperatureMaxTime":1420185600,"apparentTemperatureMin":-4.55,"apparentTemperatureMinTime":1420210800,"apparentTemperatureMax":5,"apparentTemperatureMaxTime":1420192800}]},"offset":6}'

In [99]:
json_data = json.loads(res.text)

In [100]:
keys = [x for x in json_data['daily']['data'][0].keys()]

In [101]:
len(keys)

30

In [102]:
keys

['apparentTemperatureMax',
 'precipType',
 'summary',
 'apparentTemperatureMaxTime',
 'temperatureLowTime',
 'sunriseTime',
 'temperatureHighTime',
 'sunsetTime',
 'dewPoint',
 'apparentTemperatureHighTime',
 'windBearing',
 'apparentTemperatureLow',
 'cloudCover',
 'temperatureMaxTime',
 'visibility',
 'humidity',
 'apparentTemperatureMinTime',
 'temperatureHigh',
 'windSpeed',
 'apparentTemperatureHigh',
 'icon',
 'pressure',
 'temperatureMin',
 'temperatureMax',
 'moonPhase',
 'apparentTemperatureLowTime',
 'temperatureLow',
 'temperatureMinTime',
 'apparentTemperatureMin',
 'time']